In [22]:
from nn.blocks.residuals import ResidualAdd
import torch.nn as nn
from collections import OrderedDict
import torch
from torch import Tensor

In [23]:

class ResNetBasicBlock(nn.Module):
    def __init__(self, in_features, out_features, activation: nn.Module = nn.ReLU(inplace=True)):
        super().__init__()
        self.in_features, self.out_features = in_features, out_features
        self.block = ResidualAdd(
            nn.Sequential(OrderedDict(
            {'conv1': nn.Conv2d(in_features, out_features, kernel_size=3, stride=2, padding=1, bias=False),
             'bn1': nn.BatchNorm2d(out_features),
             'act1': activation,
             'conv2': nn.Conv2d(out_features, out_features, kernel_size=3, padding=1, bias=False),
             'bn2': nn.BatchNorm2d(out_features),
            })),
             shortcut=nn.Sequential(OrderedDict({
            'conv': nn.Conv2d(in_features, out_features, kernel_size=1, stride=2, bias=False),
            'bn': nn.BatchNorm2d(out_features) if self.should_apply_shortcut else None
        }))
        )
        self.act = activation
        
    def forward(self, x):
        x = self.block(x)
        x = self.act(x)
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_features != self.out_features


In [27]:
class SEModule(nn.Module):
    def __init__(self, channel, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

In [30]:
class ResNetSEBasicBlock(ResNetBasicBlock):
    def __init__(self, in_features, out_features, reduction=16, *args, **kwargs):
        super().__init__(in_features, out_features, *args, **kwargs)
        self.block.add_module('se', SEModule(out_features))
        


In [31]:
x = torch.rand(1, 32,40,40)

res = ResNetSEBasicBlock(32, 64)

res

16


ResNetSEBasicBlock(
  (block): Residual(
    (block): Sequential(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (shortcut): Sequential(
      (conv): Conv2d(32, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (se): SEModule(
      (avg_pool): AdaptiveAvgPool2d(output_size=1)
      (fc): Sequential(
        (0): Linear(in_features=64, out_features=4, bias=False)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=4, out_features=64, bias=False)
        (3): Sigmoid()
      )
    )
  )
  (act): ReLU(inplace=Tru

In [168]:
res = ResNetBasicBlock(32, 64)
se_res = SEModule(res, features=64)

In [160]:
se_res(torch.rand(1, 32,40,40))

tensor([[[[0.0000, 0.0000, 0.6738,  ..., 0.0000, 0.3328, 0.0000],
          [0.0000, 0.0889, 0.0000,  ..., 0.0000, 0.0000, 0.5379],
          [0.0000, 0.1901, 0.3449,  ..., 0.0000, 0.6459, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.7397, 0.0000, 0.0000],
          [0.0000, 0.0000, 1.1589,  ..., 0.0000, 0.0000, 0.0182],
          [0.0411, 0.5858, 0.0000,  ..., 0.4667, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.7064, 0.0000, 0.0000],
          [0.0000, 1.2824, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.3845,  ..., 0.4785, 0.4482, 0.0000],
          ...,
          [0.0000, 0.6119, 1.6757,  ..., 0.0000, 0.0000, 0.3173],
          [0.0000, 0.0000, 0.0000,  ..., 0.0094, 0.0000, 1.1220],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0109, 0.0000]],

         [[0.0000, 0.8985, 0.0000,  ..., 0.5922, 0.0000, 0.5688],
          [0.1740, 0.7287, 0.0000,  ..., 0.0000, 0.4529, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

## ResNet

In [15]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import torch
from nn.resnet import *

In [20]:
x = torch.rand(1, 32,40,40)
block = ResNetBottleNeckConvs(32, 64)
print(block)
block(x).shape

ResNetBottleNeckConvs(
  (block): Sequential(
    (conv1): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): ReLU(inplace=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


torch.Size([1, 256, 40, 40])

In [26]:
layer = ResNetLayer(32, 64, block=ResNetBottleNeckBlock, n=2)
print(layer)
layer(x)

ResNetLayer(
  (blocks): Sequential(
    (0): ResNetBottleNeckBlock(
      (block): Residual(
        (block): ResNetBottleNeckConvs(
          (block): Sequential(
            (conv1): Conv2d(32, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act1): ReLU(inplace=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act2): ReLU(inplace=True)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (shortcut): ResNetShorcut(
          (conv): Conv2d(32, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1,

RuntimeError: The size of tensor a (256) must match the size of tensor b (64) at non-singleton dimension 1